In [ ]:
import requests
import pandas as pd
import backtrader as bt
from datetime import datetime
import random


In [ ]:
# Data fetching functions (unchanged)
def get_datetime_timestamp(date_str):
    return int(pd.to_datetime(date_str).timestamp())

def ohlcv(symbol, resolution, start, stop):
    base_url = 'https://api.nobitex.ir/market/udf/history'
    ohlc_url = f"{base_url}?symbol={symbol}&resolution={resolution}&from={start}&to={stop}"
    
    try:
        response = requests.get(ohlc_url)
        response.raise_for_status()
        data = response.json()
        
        if not data.get('t') or data['s'] != 'ok':
            raise ValueError("Invalid or empty data returned from API")
        
        ohlcv_data = pd.DataFrame({
            'DateTime': data['t'],
            'Open': data['o'],
            'High': data['h'],
            'Low': data['l'],
            'Close': data['c'],
            'Volume': data['v']
        })
        
        ohlcv_data['DateTime'] = pd.to_datetime(ohlcv_data['DateTime'], unit='s')
        ohlcv_data.to_csv('dataset.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')
        print(f"Data fetched: {len(ohlcv_data)} bars for {symbol} from {start} to {stop}")
        return ohlcv_data
    
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"Data error: {e}")
        return pd.DataFrame()
    

In [ ]:
# Leverage Sizer (unchanged)
class LeverageSizer(bt.Sizer):
    params = (
        ('leverage', 5.0),
        ('min_size', 0.0001),
        ('margin_buffer', 0.9),  # Use 90% of available cash to avoid margin issues
    )

    def _getsizing(self, comminfo, cash, data, isbuy):
        price = data.close[0]
        position_size = (cash * self.params.leverage * self.params.margin_buffer) / price
        position_size = max(position_size, self.params.min_size)
        self.log(f"Position size: {position_size:.6f}, cash={cash:.2f}, price={price:.2f}, margin_required={position_size * price / self.params.leverage:.2f}")
        return position_size

    def log(self, txt):
        print(f"Sizer: {txt}")
        

In [21]:
# Coin Flip Strategy with PnL calculation
class CoinFlipStrategy(bt.Strategy):
    params = (
        ('tp_percent', 0.015),  # 1% take profit
        ('sl_percent', 0.005),  # 0.5% stop loss
        ('leverage', 5.0),  # 5x leverage
        ('min_cash', 0.01),  # Minimum cash to trade
    )

    def __init__(self):
        self.pending_order = None
        self.entry_price = None
        self.entry_size = None
        self.is_long = None
        self.heads = 0
        self.tails = 0
        self.trade_pnls = []  # List to store PnL for each trade
        self.log("Strategy initialized")

    def log(self, txt):
        dt = self.datas[0].datetime.date(0) if self.datas else "Unknown Date"
        print(f"{dt}: {txt}")

    def notify_order(self, order):
        status_map = {
            0: "Created", 1: "Submitted", 2: "Accepted", 3: "Partial",
            4: "Completed", 5: "Canceled", 6: "Expired", 7: "Margin", 8: "Rejected"
        }
        
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.position:  # Opening a long position
                    self.log(f"BUY EXECUTED (OPEN LONG), Price: {order.executed.price:.2f}, Size: {order.executed.size:.6f}")
                    self.entry_price = order.executed.price
                    self.entry_size = order.executed.size
                    self.is_long = True
                else:  # Closing a short position
                    exit_price = order.executed.price
                    # Calculate PnL for short: (entry_price - exit_price) * size * leverage
                    pnl = (self.entry_price - exit_price) * self.entry_size * self.params.leverage
                    self.trade_pnls.append(pnl)
                    self.log(f"BUY EXECUTED (CLOSE SHORT), Price: {exit_price:.2f}, Size: {order.executed.size:.6f}, PnL: {pnl:.2f}")
            elif order.issell():
                if self.position:  # Opening a short position
                    self.log(f"SELL EXECUTED (OPEN SHORT), Price: {order.executed.price:.2f}, Size: {order.executed.size:.6f}")
                    self.entry_price = order.executed.price
                    self.entry_size = order.executed.size
                    self.is_long = False
                else:  # Closing a long position
                    exit_price = order.executed.price
                    # Calculate PnL for long: (exit_price - entry_price) * size * leverage
                    pnl = (exit_price - self.entry_price) * self.entry_size * self.params.leverage
                    self.trade_pnls.append(pnl)
                    self.log(f"SELL EXECUTED (CLOSE LONG), Price: {exit_price:.2f}, Size: {order.executed.size:.6f}, PnL: {pnl:.2f}")
            self.pending_order = None
            self.log("Pending order cleared")

        elif order.status in [order.Canceled, order.Margin, order.Rejected, order.Expired]:
            self.log(f"Order Failed: Status={status_map.get(order.status, 'Unknown')}, Size={order.size:.6f}, Price={order.price:.2f}")
            self.pending_order = None
            self.log("Pending order cleared due to failure")

    def next(self):
        if self.pending_order or self.position:
            return

        price = self.data.close[0]
        cash = self.broker.get_cash()

        if cash < self.params.min_cash:
            self.log(f"Not enough cash to trade: {cash:.2f}")
            return

        flip = random.choice(['head', 'tail'])
        if flip == 'head':
            self.heads += 1
            tp = price * (1 + self.params.tp_percent)
            sl = price * (1 - self.params.sl_percent)
            self.pending_order = self.buy_bracket(
                price=price,
                limitprice=tp,
                stopprice=sl,
                exectype=bt.Order.Market
            )
            self.log(f"HEAD → BUY ORDER PLACED: Price={price:.2f}, TP={tp:.2f}, SL={sl:.2f}")
        else:
            self.tails += 1
            tp = price * (1 - self.params.tp_percent)
            sl = price * (1 + self.params.sl_percent)
            self.pending_order = self.sell_bracket(
                price=price,
                limitprice=tp,
                stopprice=sl,
                exectype=bt.Order.Market
            )
            self.log(f"TAIL → SELL ORDER PLACED: Price={price:.2f}, TP={tp:.2f}, SL={sl:.2f}")
            

In [31]:
if __name__ == '__main__':
    # Fetch data
    start = get_datetime_timestamp('2020-04-20')
    stop = get_datetime_timestamp('2025-04-25')
    df = ohlcv('btcusdt', 1, start, stop)
    
    if df.empty:
        print("No data fetched, exiting")
        exit(1)

    # Prepare data
    df = pd.read_csv('dataset.csv')
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df.set_index('DateTime', inplace=True)
    
    data = bt.feeds.PandasData(
        dataname=df,
        timeframe=bt.TimeFrame.Minutes,
        openinterest=-1
    )
    
    # Setup Cerebro
    cerebro = bt.Cerebro()
    cerebro.addstrategy(CoinFlipStrategy)
    cerebro.adddata(data)
    cerebro.broker.setcash(1000.0)
    cerebro.broker.setcommission(commission=0.001, margin=0.2)  # 20% margin requirement (1/leverage)
    cerebro.addsizer(LeverageSizer)

    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')

    # Run backtest
    results = cerebro.run()
    strat = results[0]
    

Data fetched: 40000 bars for btcusdt from 1587340800 to 1745539200
2025-04-24: Strategy initialized
Sizer: Position size: 0.052278, cash=1000.00, price=86079.00, margin_required=900.00
2025-03-28: TAIL → SELL ORDER PLACED: Price=86079.00, TP=84787.82, SL=86509.39
2025-03-28: SELL EXECUTED (OPEN SHORT), Price: 86078.90, Size: -0.052278
2025-03-28: Pending order cleared
2025-03-28: BUY EXECUTED (CLOSE SHORT), Price: 84787.82, Size: 0.052278, PnL: -337.47
2025-03-28: Pending order cleared
2025-03-28: Order Failed: Status=Canceled, Size=0.052278, Price=86509.39
2025-03-28: Pending order cleared due to failure
Sizer: Position size: 0.056677, cash=1067.49, price=84756.00, margin_required=960.75
2025-03-28: TAIL → SELL ORDER PLACED: Price=84756.00, TP=83484.66, SL=85179.78
2025-03-28: SELL EXECUTED (OPEN SHORT), Price: 84756.00, Size: -0.056677
2025-03-28: Pending order cleared
2025-03-29: BUY EXECUTED (CLOSE SHORT), Price: 83484.66, Size: 0.056677, PnL: -360.28
2025-03-29: Pending order clea

In [32]:
# Print results
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")
print(f"Sharpe Ratio: {strat.analyzers.sharpe.get_analysis()}")
print(f"SQN: {strat.analyzers.sqn.get_analysis()}")
print(f"Drawdown: {strat.analyzers.drawdown.get_analysis()}")
print("Trade Analyzer:")
for key, value in strat.analyzers.trades.get_analysis().items():
    print(f"  {key}: {value}")
print(f"Heads: {strat.heads}, Tails: {strat.tails}")

# Print PnL for each trade
print("\nTrade PnL Summary:")
if strat.trade_pnls:
    for i, pnl in enumerate(strat.trade_pnls, 1):
        print(f"Trade {i}: PnL = {pnl:.2f}")
    print(f"Total PnL: {sum(strat.trade_pnls):.2f}")
    print(f"Average PnL: {sum(strat.trade_pnls) / len(strat.trade_pnls):.2f}")
else:
    print("No trades executed.")
    

Final Portfolio Value: 330.71
Sharpe Ratio: OrderedDict({'sharperatio': None})
SQN: AutoOrderedDict({'sqn': -1.6394603659091427, 'trades': 285})
Drawdown: AutoOrderedDict({'len': 37325, 'drawdown': 78.42374560442528, 'moneydown': 1202.0533434975541, 'max': AutoOrderedDict({'len': 37325, 'drawdown': 85.7907821639898, 'moneydown': 1314.972853524037})})
Trade Analyzer:
  total: AutoOrderedDict({'total': 286, 'open': 1, 'closed': 285})
  streak: AutoOrderedDict({'won': AutoOrderedDict({'current': 3, 'longest': 6}), 'lost': AutoOrderedDict({'current': 0, 'longest': 18})})
  pnl: AutoOrderedDict({'gross': AutoOrderedDict({'total': -672.8107147887201, 'average': -2.36073935013586}), 'net': AutoOrderedDict({'total': -672.8282190240666, 'average': -2.360800768505497})})
  won: AutoOrderedDict({'total': 67, 'pnl': AutoOrderedDict({'total': 2237.3554869497016, 'average': 33.39336547686122, 'max': 97.90145634654597})})
  lost: AutoOrderedDict({'total': 218, 'pnl': AutoOrderedDict({'total': -2910.1